# 1. If the different directions use a different number of hidden units, how will the shape of $H_t$ change?

$H_t.shape[-1] = \overrightarrow{H_t}.shape[-1] + \overleftarrow{H_t}.shape[-1]$

In [13]:
import sys
import torch.nn as nn
import torch
import warnings
sys.path.append('/home/jovyan/work/d2l_solutions/notebooks/exercises/d2l_utils/')
import d2l
from torchsummary import summary
warnings.filterwarnings("ignore")
from sklearn.model_selection import ParameterGrid

class BiRNNScratch(d2l.Module):
    def __init__(self, num_inputs, num_hiddens, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.f_rnn = d2l.RNNScratch(num_inputs, num_hiddens[0], sigma)
        self.b_rnn = d2l.RNNScratch(num_inputs, num_hiddens[1], sigma)
        self.num_hiddens = sum(num_hiddens)  # The output dimension will be doubled
        
    def forward(self, inputs, Hs=None):
        f_H, b_H = Hs if Hs is not None else (None, None)
        f_outputs, f_H = self.f_rnn(inputs, f_H)
        b_outputs, b_H = self.b_rnn(reversed(inputs), b_H)
        outputs = [torch.cat((f, b), -1) for f, b in zip(
            f_outputs, reversed(b_outputs))]
        return outputs, (f_H, b_H)

In [14]:
bi_rnn = BiRNNScratch(num_inputs=4, num_hiddens=[8,16])
X = torch.randn(4)
outputs = BiRNNScratch(X)
outputs.shape

AttributeError: module 'd2l' has no attribute 'RNNScratch'

# 2. Design a bidirectional RNN with multiple hidden layers.

# 3. Polysemy is common in natural languages. For example, the word “bank” has different meanings in contexts “i went to the bank to deposit cash” and “i went to the bank to sit down”. How can we design a neural network model such that given a context sequence and a word, a vector representation of the word in the correct context will be returned? What type of neural architectures is preferred for handling polysemy?